In [1]:
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error,r2_score
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
import argparse
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time
from sklearn.pipeline import  Pipeline, make_pipeline

from transformation import RemoveSkewnessKurtosis, Standardize, ImputeNumeric


# Import Data
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
X_admission1 = df1.drop(columns = ['readmission within 300 days', 'died_within_900days'])
Y_admission1 = df1[['readmission within 300 days']]

X_mortality1 = df1.drop(columns = ['died_within_900days'])
Y_mortality1 = df1[['died_within_900days']]

# Split Train and Test
X_train_ad1, X_test_ad1, y_train_ad1, y_test_ad1 = train_test_split(X_admission1, Y_admission1, test_size=0.20, random_state=42)

transform_steps = [("ImputeNumeric", ImputeNumeric()),
                   ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
                   ('StandardizeStandardScaler', Standardize(RobustScaler()))]
transform_pipeline = Pipeline(transform_steps)

X_train_ad1 = transform_pipeline.transform(X_train_ad1)

/home/vivi/FDA/src/models/transformation.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = self.scalar.fit_transform(X[i].values.reshape(-1,1))


In [5]:
# Grid Search CV
import numpy as np
import pandas as pd
import scipy as sp
import copy,os,sys,psutil
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
 
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
 
def print_best_score(gsearch,param_test):
    print("Best score: %0.3f" % gsearch.best_score_)
    print("Best parameters set:")
    for param_name in sorted(param_test.keys()):
        print("\t%s: %r" % (param_name,gsearch.best_params_[param_name]))
 
LogisticRegression_param = { 
    'penalty' : ['l1','l2']
    }

def LogisticRegression_Grid_CV(X_train,y_train, LogisticRegression_param):
    estimator = LogisticRegression()
    gsearch = GridSearchCV(estimator , param_grid = LogisticRegression_param, scoring='roc_auc', cv=5)
    gsearch.fit(X_train, y_train)
    #gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_
    print_best_score(gsearch,LogisticRegression_param)

LogisticRegression_Grid_CV(X_train_ad1, y_train_ad1, LogisticRegression_param)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STO

Best score: 0.778
Best parameters set:
	penalty: 'l2'
